In [69]:
# 빅쿼리 데이터를 불러오기위한 쿼리
query = '''
# 위경도를 간략화 하기위한 db 테이블
with db as (
    select taxi.*,
  					# taxi의 실제 위경도 데이터를 zipcode에 해당하는 위경도 데이터로 바꿔서 사용합니다.
 						# internal_point_lat은 특정 zipcode의 위도입니다.
            pick.internal_point_lat as pickup_zip_code_lat,
            pick.internal_point_lon as pickup_zip_code_lon,
            drop.internal_point_lat as dropoff_zip_code_lat,
            drop.internal_point_lon as dropoff_zip_code_lon
    from (
        select *
        from `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
            # 1월 1일의 데이터만 필터링 합니다.
			where extract (date from pickup_datetime) = '2015-01-01'
      		# 위도는 -90에서 90까지만 유효합니다. 그 외의 데이터는 오류이므로 제외합니다.
            and pickup_latitude between -90 and 90
            and dropoff_latitude between -90 and 90
    ) as taxi # 실제 운영 기록이 담긴 taxi테이블
        join (
            select * 
            from `bigquery-public-data.geo_us_boundaries.zip_codes`
          	# 뉴욕시의 데이터만 필터링, 뉴욕 외의 지역에서 픽업한 데이터는 제외됩니다.
            where state_code = 'NY'
        ) as pick # 실제 픽업 위치의 위경도를 zipcode의 대표 위경도로 단일화시킬 pick테이블(join후 테이블 치환)
  					# taxi테이블의 위경도가 어느 zipcode에 해당되는지 찾아서 join시키기 위한 부분
            on st_contains(pick.zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
        join (
            select *
            from `bigquery-public-data.geo_us_boundaries.zip_codes`
            where state_code = 'NY'
        ) as drop # 실제 드랍 위치의 위경도를 치환할 drop테이블
            on st_contains(drop.zip_code_geom, st_geogpoint(dropoff_longitude, dropoff_latitude))
)
select datetime_trunc(pickup_datetime, hour) as pickup_hour, # 시간대별로 집계
    # 픽업/드랍 위경도: zipcode별로 단일화되어서 집계가 가능해짐
    pickup_zip_code_lat,
    pickup_zip_code_lon,
    dropoff_zip_code_lat,
    dropoff_zip_code_lon,
    count(*) as cnt
from db
# 시간, 픽업/드랍 위경도별로 그룹바이
group by 1,2,3,4,5
# 집계 후 가운트가 적은 경우는 필터링
having cnt >= 30
'''

In [70]:
# gbq로 빅쿼리 데이터 임포트
df = pd.read_gbq(query = query, dialect = 'standard',project_id = "public-data-projects")

In [71]:
# 빅쿼리에서 시간별, 위경도별로 집계된 데이터
df.tail()

,pickup_hour,pickup_zip_code_lat,pickup_zip_code_lon,dropoff_zip_code_lat,dropoff_zip_code_lon,cnt
2426,2015-01-01 11:00:00,40.726399,-73.978634,40.731829,-73.989181,31
2427,2015-01-01 20:00:00,40.765829,-73.987341,40.755332,-73.993139,40
2428,2015-01-01 03:00:00,40.750636,-73.997177,40.742043,-74.000620,55
2429,2015-01-01 12:00:00,40.745221,-73.978294,40.742043,-74.000620,33
2430,2015-01-01 18:00:00,40.725581,-73.998078,40.745221,-73.978294,35


In [82]:
import pydeck as pdk

layer1 = pdk.Layer(
    'ArcLayer', # 사용할 레이어 타입
    df,
    get_source_position='[pickup_zip_code_lon, pickup_zip_code_lat]', # 출발 포지션
    get_target_position='[dropoff_zip_code_lon, dropoff_zip_code_lat]', #도착 포지션
    get_source_color= [0, 255, 0, 40],
    get_target_color= [240, 100, 0, 40],
    width_units='meters',
    get_width="cnt",
    pickable= True,
    auto_highlight=True,
)

nyc_center = [-73.98, 40.76] 
view_state = pdk.ViewState(longitude=nyc_center[0], 
                           latitude=nyc_center[1], zoom=11, pitch=35)


In [83]:
r = pdk.Deck(layers=[layer1], initial_view_state=view_state,
             tooltip={
                 'html': '<b>count:</b> {cnt}',
                 'style': {
                     'color': 'white'
                 }
             }
            )
r.picking_radius = 5000

r.show()


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…